In [0]:
import pandas as pd

# 数据准备、清洗部分

首先读取数据

In [0]:
hd_df = pd.read_csv('hair_dryer.tsv', sep='\t')
mw_df = pd.read_csv('microwave.tsv', sep='\t')
pf_df = pd.read_csv('pacifier.tsv', sep='\t')

df = [hd_df, mw_df, pf_df]

In [3]:
for dataset in df:
  print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11470 entries, 0 to 11469
Data columns (total 15 columns):
marketplace          11470 non-null object
customer_id          11470 non-null int64
review_id            11470 non-null object
product_id           11470 non-null object
product_parent       11470 non-null int64
product_title        11470 non-null object
product_category     11470 non-null object
star_rating          11470 non-null int64
helpful_votes        11470 non-null int64
total_votes          11470 non-null int64
vine                 11470 non-null object
verified_purchase    11470 non-null object
review_headline      11468 non-null object
review_body          11470 non-null object
review_date          11470 non-null object
dtypes: int64(5), object(10)
memory usage: 1.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615 entries, 0 to 1614
Data columns (total 15 columns):
marketplace          1615 non-null object
customer_id          1615 non-null int64
revie

可以查看一下基本数学信息

In [4]:
for dataset in df:
  print(dataset.describe())

        customer_id  product_parent   star_rating  helpful_votes   total_votes
count  1.147000e+04    1.147000e+04  11470.000000   11470.000000  11470.000000
mean   2.815122e+07    4.846338e+08      4.116042       2.179076      2.563296
std    1.523877e+07    2.873240e+08      1.300333      14.241304     15.382528
min    1.246400e+04    4.239600e+05      1.000000       0.000000      0.000000
25%    1.491441e+07    2.351060e+08      4.000000       0.000000      0.000000
50%    2.707123e+07    4.867740e+08      5.000000       0.000000      0.000000
75%    4.233644e+07    7.322523e+08      5.000000       1.000000      1.000000
max    5.309637e+07    9.994366e+08      5.000000     499.000000    575.000000
        customer_id  product_parent  star_rating  helpful_votes  total_votes
count  1.615000e+03    1.615000e+03  1615.000000    1615.000000  1615.000000
mean   2.956669e+07    5.327655e+08     3.444582       5.621672     6.669350
std    1.551920e+07    2.312119e+08     1.645238      27.7

In [5]:
combine = pd.concat(df, axis=0)
combine['product_category'] = combine['product_category'].map( {'Major Appliances': 'Microwave', 'major appliances': 'Microwave', 'Baby': 'Pacifier', 'baby': 'Pacifier', 
                                             'Beauty': 'HairDryer'} ).astype(str)

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

plt.rcParams['figure.figsize'] = (15, 9)
plt.style.use('fivethirtyeight')

sns.boxenplot(combine['product_category'], combine['star_rating'], palette = 'spring')
plt.title("Category vs Ratings")
plt.xticks(rotation = 90)
plt.show()

发现其实不同类别商品之间的信息差别挺大的，(mean&std)，所以打算分开建模

查看一下是否有NaN数据

In [6]:
for dataset in df:
  print(dataset.isnull().any())

marketplace          False
customer_id          False
review_id            False
product_id           False
product_parent       False
product_title        False
product_category     False
star_rating          False
helpful_votes        False
total_votes          False
vine                 False
verified_purchase    False
review_headline       True
review_body          False
review_date          False
dtype: bool
marketplace          False
customer_id          False
review_id            False
product_id           False
product_parent       False
product_title        False
product_category     False
star_rating          False
helpful_votes        False
total_votes          False
vine                 False
verified_purchase    False
review_headline      False
review_body          False
review_date          False
dtype: bool
marketplace          False
customer_id          False
review_id            False
product_id           False
product_parent       False
product_title        False
prod

发现hair dryer和pacifier均有缺省值。我们仔细看一下这些数据

In [7]:
print(hd_df[hd_df['review_headline'].isnull()])
print(pf_df[pf_df['review_body'].isnull()])

     marketplace  ...  review_date
273           US  ...     8/9/2015
2433          US  ...    2/17/2015

[2 rows x 15 columns]
     marketplace  customer_id  ... review_body review_date
950           US     38484571  ...         NaN   7/17/2015
3456          US     45456400  ...         NaN   3/30/2015

[2 rows x 15 columns]


发现缺省的数据并不多，所以决定此处直接删除。

In [0]:
hd_df = hd_df.dropna()
pf_df = pf_df.dropna()

接下来分析一下数据的特点。

因为helpful_votes不好量化，因此这里对total_votes大于2的数据进行归一化，变成helpful_rates，即：

$$ helpful\_rating = 4* [\frac{helpful\_votes}{total\_votes}] +1$$

而小于2的数据均填充为NaN

In [9]:
helpfulrates = round(4*hd_df[hd_df['total_votes']>2]['helpful_votes']/hd_df[hd_df['total_votes']>2]['total_votes'])+1
hd_df['helpful_rating'] = helpfulrates

helpfulrates = round(4*mw_df[mw_df['total_votes']>2]['helpful_votes']/mw_df[mw_df['total_votes']>2]['total_votes'])+1
mw_df['helpful_rating'] = helpfulrates

helpfulrates = round(4*pf_df[pf_df['total_votes']>2]['helpful_votes']/pf_df[pf_df['total_votes']>2]['total_votes'])+1
pf_df['helpful_rating'] = helpfulrates

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  




* 地区数据是否相同？




In [10]:
for dataset in df:
  print(dataset['marketplace'].unique())

['US']
['US' 'us']
['US' 'us']


In [11]:
print(hd_df['product_id'].unique().shape)
print(hd_df['product_parent'].unique().shape)

print(mw_df['product_id'].unique().shape)
print(mw_df['product_parent'].unique().shape)

print(pf_df['product_id'].unique().shape)
print(pf_df['product_parent'].unique().shape)

(538,)
(473,)
(80,)
(55,)
(6482,)
(5432,)


*   发现都是在US购买的，可以认为该项冗余，可以删除。
*   删除customer_id、product_parent、review_id、product_category、review_headline

In [0]:
hd_df = hd_df.drop(columns=['marketplace','customer_id','product_id','review_id','product_category','product_title','total_votes','helpful_votes','review_headline'])
mw_df = mw_df.drop(columns=['marketplace','customer_id','product_id','review_id','product_category','product_title','total_votes','helpful_votes','review_headline'])
pf_df = pf_df.drop(columns=['marketplace','customer_id','product_id','review_id','product_category','product_title','total_votes','helpful_votes','review_headline'])

将vine、verified_purchase转换成数值形式

In [13]:
df = [hd_df, mw_df, pf_df]
for dataset in df:
  print(dataset['vine'].unique())
  print(dataset['verified_purchase'].unique())

['N' 'Y']
['Y' 'N']
['N' 'n' 'Y']
['Y' 'N' 'y' 'n']
['N' 'n' 'y' 'Y']
['Y' 'y' 'n' 'N']


In [14]:
for dataset in df:
    dataset['vine'] = dataset['vine'].map( {'Y': 1, 'y':1, 'N': 0, 'n': 0} ).astype(int)
    dataset['verified_purchase'] = dataset['verified_purchase'].map( {'Y': 1, 'y':1, 'N': 0, 'n': 0} ).astype(int)

hd_df.head()

product_parent  star_rating  ...  review_date  helpful_rating
0       732252283            5  ...    8/31/2015             NaN
1       670161917            4  ...    8/31/2015             NaN
2        16483457            5  ...    8/31/2015             NaN
3       253917972            5  ...    8/31/2015             NaN
4       732252283            4  ...    8/31/2015             NaN

[5 rows x 7 columns]

将处理好后的数据保存为csv格式

In [0]:
hd_df.to_csv('hd.csv', index = False, index_label=None)
mw_df.to_csv('mw.csv', index = False, index_label=None)
pf_df.to_csv('pf.csv', index = False, index_label=None)